#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np

#### 2. Import BigQuery library


In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./ironhack_service_account_big_query.json"
from google.cloud import bigquery

#### 3. Create a connection to BigQuery

In [3]:
client = bigquery.Client()

#### 4. Import data from the users table from the stackoverflow public database for United Kingdom only

In [4]:
sql='''
SELECT
    *
FROM 
    `bigquery-public-data.stackoverflow.users`
WHERE
    location = "United Kingdom"
'''
query_job = client.query(query = sql)
df = query_job.to_dataframe()
#df.head()

#### 5. Rename id column to user_id

In [5]:
data = df.rename(columns={'id':'user_id'})
data.head()

,user_id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,2434,Daniel James,None,None,2008-08-22 10:14:24.370000+00:00,2019-09-02 18:09:02.960000+00:00,United Kingdom,3659,333,24,469,None,None
1,21442,nationale,None,None,2008-09-24 00:03:25.460000+00:00,2008-10-13 02:26:37.530000+00:00,United Kingdom,183,2,0,25,None,None
2,35186,Rav,None,None,2008-11-06 17:26:38.160000+00:00,2019-11-30 16:51:11.733000+00:00,United Kingdom,587,0,4,46,https://i.stack.imgur.com/ykhD5.jpg,None
3,95361,Russ Hayward,<p>I write code primarily in Java but I enjoy ...,None,2009-04-24 08:38:24.340000+00:00,2019-11-29 11:11:07.363000+00:00,United Kingdom,5369,679,16,793,None,http://www.blanksite.com/
4,134189,jezmck,None,None,2009-05-14 19:33:12.667000+00:00,2019-11-29 13:38:48.700000+00:00,United Kingdom,918,507,73,89,https://i.stack.imgur.com/xPsDD.jpg?s=128&g=1,http://jezmck.com


#### 6. Import from the stackoverflow_posts table all items which have up to 20 view counts and only select 'id', 'owner_user_id', 'score', 'view_count', 'comment_count', 'favorite_count'

In [6]:
sql_2='''
SELECT
    id
    ,owner_user_id
    ,score
    ,view_count
    ,comment_count
    ,favorite_count
FROM 
    `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE
    view_count <= 20
;'''
query_job = client.query(query = sql_2)
df_2 = query_job.to_dataframe()

#df_2.head()



#### 7. Rename id column to post_id and owner_user_id to user_id

In [7]:
data_2 = df_2.rename(columns={'id':'post_id',
                             "owner_user_id" : "user_id"})
data_2.head()

,post_id,user_id,score,view_count,comment_count,favorite_count
0,23154134,NaN,1,20,0,NaN
1,37516574,NaN,0,7,0,NaN
2,37319831,NaN,1,18,0,NaN
3,31140287,NaN,1,16,3,NaN
4,36546336,NaN,0,8,0,NaN


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [8]:
users_cols = data[["user_id", "reputation", "views", "up_votes", "down_votes"]]
users_cols.head()

,user_id,reputation,views,up_votes,down_votes
0,2434,3659,469,333,24
1,21442,183,25,2,0
2,35186,587,46,0,4
3,95361,5369,793,679,16
4,134189,918,89,507,73


In [11]:
post_cols = data_2[["post_id", "score", "user_id", "view_count", "comment_count"]]
post_cols.head()

,post_id,score,user_id,view_count,comment_count
0,23154134,1,NaN,20,0
1,37516574,0,NaN,7,0
2,37319831,1,NaN,18,0
3,31140287,1,NaN,16,3
4,36546336,0,NaN,8,0


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [12]:
users_cols = users_cols.merge(post_cols, left_on='user_id', right_on='user_id')
users_cols.head()

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,1076743,7146,1177,450,120,33428716,0,17,0
1,1076743,7146,1177,450,120,15036385,0,16,0
2,1864489,550,185,194,3,37754600,0,20,2
3,2508203,75,24,6,0,37727168,0,15,0
4,117859,1384,170,91,6,19060940,0,20,0


#### 9. How many missing values do you have in your merged dataframe? 

In [14]:
users_cols.isnull().sum()

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

In [13]:
users_cols.isnull().sum().sum()

0

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [22]:
users_cols = users_cols.drop("post_id", axis=1)

In [29]:
low_variance = []

for col in users_cols._get_numeric_data():
    minimum = min(users_cols[col])
    ninety_per = np.percentile(users_cols[col], 90)
    if ninety_per == minimum:
        low_variance.append(col)
users_cols = users_cols.drop(low_variance, axis=1)

In [30]:
stats = users_cols.describe().transpose()
stats["IQR"] = stats["75%"] - stats["25%"]
stats

,count,mean,std,min,25%,50%,75%,max,IQR
user_id,1078.0,2.250329e+06,1.824147e+06,274.0,653292.0,1937858.0,3494103.75,6449207.0,2840811.75
reputation,1078.0,7.333054e+03,2.368506e+04,1.0,231.0,1106.0,5229.75,279695.0,4998.75
views,1078.0,6.256716e+02,1.637005e+03,0.0,39.0,139.5,495.75,27257.0,456.75
up_votes,1078.0,3.883145e+02,7.119623e+02,0.0,16.0,94.5,385.75,7845.0,369.75
down_votes,1078.0,5.051670e+01,2.028212e+02,0.0,0.0,3.0,20.00,3624.0,20.00
score,1078.0,1.270872e-01,5.873860e-01,-7.0,0.0,0.0,0.00,4.0,0.00
view_count,1078.0,1.558256e+01,3.709026e+00,3.0,13.0,16.0,19.00,20.0,6.00
comment_count,1078.0,9.202226e-01,1.632748e+00,0.0,0.0,0.0,1.00,11.0,1.00


In [27]:
outliers = pd.DataFrame(columns = users_cols.columns)

for col in stats.index:
    iqr = stats.at[col, "IQR"]
    cutoff = iqr * 1.5
    lower = stats.at[col, "25%"] - cutoff
    upper = stats.at[col, "75%"] + cutoff
    results = users_cols[(users_cols[col] < lower) |
                  (users_cols[col] > upper)].copy()
    results["Outlier"] = col
    outliers = outliers.append(results)

In [37]:
outliers

,Outlier,comment_count,down_votes,reputation,score,up_votes,user_id,view_count,views
6,reputation,0,32,24921,0,2970,33051,15,3241
32,reputation,0,198,37327,0,1900,38522,13,2872
33,reputation,1,198,37327,0,1900,38522,19,2872
34,reputation,2,198,37327,0,1900,38522,18,2872
35,reputation,0,198,37327,0,1900,38522,7,2872
...,...,...,...,...,...,...,...,...,...
1026,comment_count,5,28,5466,0,16,3195915,15,101
1051,comment_count,3,0,47,0,2,1542051,20,1
1054,comment_count,4,0,43,-7,3,6118448,14,5
1063,comment_count,3,0,61,0,0,3054080,17,13


In [38]:
outliers.to_csv("./outliers.csv", index = False)